In [1]:
from flo_ai import Flo
from flo_ai import FloSession
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.tools.tavily_search.tool import TavilySearchResults

llm = ChatOpenAI(temperature=0, model_name='gpt-4o-mini')
session = FloSession(llm)

session.register_tool(
    name="InternetSearchTool", 
    tool=TavilySearchResults()
)

In [3]:
simple_weather_checking_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: weather-assistant
agent:
    name: Weather Assistant
    job: >
      Given the city name you are capable of answering the latest whether this time of the year by searching the internet
    tools:
      - name: InternetSearchTool
"""

In [4]:
flo = Flo.build(session, simple_weather_checking_agent)

print(flo.invoke("Whats the whether in california"))



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'California weather October 2023'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'California City', 'region': 'California', 'country': 'United States of America', 'lat': 35.13, 'lon': -117.99, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1725351692, 'localtime': '2024-09-03 01:21'}, 'current': {'last_updated_epoch': 1725351300, 'last_updated': '2024-09-03 01:15', 'temp_c': 28.1, 'temp_f': 82.6, 'is_day': 0, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/night/116.png', 'code': 1003}, 'wind_mph': 8.1, 'wind_kph': 13.0, 'wind_degree': 190, 'wind_dir': 'S', 'pressure_mb': 1015.0, 'pressure_in': 29.98, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 25, 'cloud': 50, 'feelslike_c': 27.0, 'feelslike_f': 80.6, 'windchill_c': 24.9, 'windchill_f': 76.8, 'heatindex_c': 24.7, 'heatindex_f': 76.4, 'dewpoint_c': 5.8, 'dewpoint_f': 4

In [5]:
simple_ll_agent = """
apiVersion: flo/alpha-v1
kind: FloAgent
name: llm-assistant
agent:
    name: Ask llm anything
    kind: llm
    job: >
      You are a high school maths teacher. Answer any questions the students ask 
"""

In [6]:
flo = Flo.build(session, simple_ll_agent)

print(flo.invoke("What is pythagorus theorum"))



> Entering new AgentExecutor chain...


BadRequestError: Error code: 400 - {'error': {'message': "Invalid 'tools': empty array. Expected an array with minimum length 1, but got an empty array instead.", 'type': 'invalid_request_error', 'param': 'tools', 'code': 'empty_array'}}